In [1]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import scipy.stats as scs
%matplotlib inline

matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [ ]:
data = pd.read_csv("/mnt/c/users/ado3iz/downloads/CC-P.csv")

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
btc = data[data['from_symbol'] == "BTC"]
btc = btc[btc['to_symbol'] == "JPY"]

In [ ]:
btc

In [ ]:
from datetime import datetime

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
#lambda x: x = datetime.utcfromtimestamp(btc['unix_time']).strftime('%Y-%m-%d %H:%M:%S')

# test = btc['unix_time'].apply
btc['unix_time'] = btc['unix_time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
btc['unix_time'] = pd.to_datetime(btc['unix_time'])

In [ ]:
btc

In [ ]:
import pandas_profiling as pp

pp.ProfileReport(btc)



In [ ]:
btc.drop(['volume_from', 'volume_to', 'date', 'time', 'from_symbol', 'to_symbol'], axis = 1, inplace = True)

In [ ]:
btc = btc.reset_index(drop=True)

In [ ]:
btc.drop(['index'], axis = 1, inplace = True)

In [ ]:
btc.index = btc['unix_time']

In [ ]:
btc.drop(['unix_time'], axis = 1, inplace = True)

In [ ]:
btc['close'].plot()

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
from numpy import array
n_steps = 5
# split into samples
X, y = split_sequence(btc['close'], n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

In [ ]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import ConvLSTM2D
n_seq = 2
n_steps = 5
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)
# # demonstrate prediction
# x_input = array([70, 80, 90])
# x_input = x_input.reshape((1, n_steps, n_features))
# yhat = model.predict(x_input, verbose=0)
# print(yhat)

In [ ]:
x_input = X[1:100]

In [ ]:
x_input.shape

In [ ]:
x_input = X[1:100]
x_input = x_input.reshape((99, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


In [ ]:
yhat = pd.DataFrame(yhat)

In [ ]:
actual = pd.DataFrame(y[1:100])

In [ ]:
yhat.columns = ['predict']
actual.columns = ['actual']

In [ ]:
actual.index

In [ ]:
polat = pd.merge(actual, yhat, left_index=True, right_index=True)
plt.figure()
polat.plot()

In [ ]:
# univariate multi-step encoder-decoder lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
#raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 5, 1
# split into samples
X, y = split_sequence(btc['close'], n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=100, verbose=0)
# demonstrate prediction
x_input = X[1:100]
x_input = x_input.reshape((99, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

In [ ]:
yhat = np.concatenate(yhat)

In [ ]:
actual = np.concatenate(y[1:100])

In [ ]:
actual['idx'] = actual.index

In [ ]:
actual['idx'] = actual['idx']+1

In [ ]:
actual.index = actual['idx']

In [ ]:
actual.loc[0] = actual.loc[1]

In [ ]:
actual.drop(['idx'], axis=1, inplace = True)

In [ ]:
yhat.loc[99] = yhat.loc[98]

In [ ]:
actual = actual.sort_index(ascending=True)

In [ ]:
yhat = pd.DataFrame(yhat)
actual = pd.DataFrame(actual)
yhat.columns = ['predict']
actual.columns = ['actual']
polat = pd.merge(actual, yhat, left_index=True, right_index=True)
plt.figure()
polat.plot()